# Implementación de algoritmo _CVRP_ en simulación generada

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

### Importar datos de simulación

In [2]:
simulacion_clientes = pd.read_csv("simulacion_clientes.csv")
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.527082,1,0,0,0,...,0,0,0,0,0,0,0.017079,"(25.68654935, -100.2772068)",25.686549,-100.277207
1,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.748310,2,0,0,0,...,0,0,0,0,0,0,0.290464,"(25.763761, -100.408254)",25.763761,-100.408254
2,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.869673,2,0,0,0,...,0,0,0,0,0,0,0.001022,"(25.79072633, -100.40731977)",25.790726,-100.407320
3,Monterrey,PORTAL DEL FRAILE,66064,FRAY JOSE,729.0,0.628113,1,0,0,0,...,0,0,0,0,0,0,0.214957,"(25.8003649, -100.4111165)",25.800365,-100.411117
4,Monterrey,CUMBRES 3 SEC,64610,HACIENDA DEL CA,310.0,0.485124,1,0,0,0,...,0,0,0,0,0,0,0.001254,"(25.743958, -100.406852)",25.743958,-100.406852


In [3]:
nuevas_coordenadas = []
for i in range(simulacion_clientes.shape[0]):
    coord =  (simulacion_clientes.iloc[i,4770],simulacion_clientes.iloc[i,4771])
    nuevas_coordenadas.append(coord)

simulacion_clientes["Coordenadas"] = nuevas_coordenadas

In [4]:
simulacion_clientes["Coordenadas"][0]

(25.68654935, -100.2772068)

In [5]:
distancia = str(geodesic(simulacion_clientes.iloc[0,4769], simulacion_clientes.iloc[1,4769]).km)
print("Distancia entre dos puntos",distancia)

Distancia entre dos puntos 15.687694259452297


In [6]:
def generar_distancias(clientes):
    matriz_distancias = []

    for i in range(clientes.shape[0]):
        coord1 = clientes.iloc[i,4769]
        distancias =  []

        for j in range(clientes.shape[0]):
            if j != i:
                coord2 = clientes.iloc[j,4769]
                distancia = float(geodesic(coord1, coord2).km)
                distancias.append(distancia)
            else:
                distancias.append(0)

        matriz_distancias.append(distancias)


    return matriz_distancias


In [7]:
distancias = generar_distancias(simulacion_clientes)
distancias

[[0,
  15.687694259452297,
  17.425235346704834,
  18.4256355946974,
  14.482148967526198,
  4.158712351046546,
  4.45384399115749,
  2.7064864991529682,
  8.884973516563253,
  11.780051252648578,
  12.242416540282992,
  31.125631900206898,
  3.5959752040520296,
  8.575477998292007,
  2.931099660463862,
  10.661816882223038,
  3.612123450124999,
  11.504135607269605,
  4.158712351046546,
  14.734524562221413,
  7.880297075831406,
  4.873863090651064,
  7.350549318555193,
  14.970909393289288,
  6.129437001877262,
  8.575477998292007,
  16.110885208348233,
  14.927158034607281,
  9.47071062686737,
  2.644012132933684,
  6.786741679950096,
  11.86813313145062,
  9.396107040478107,
  2.7426659383072747,
  12.698524309875234,
  3.7589546342524955,
  9.23648877204979,
  11.842166611275092,
  4.853739798270574,
  7.757456241815392,
  5.6898205465454375,
  31.125631900206898,
  10.033051627820866,
  1.5526654227459862,
  13.662461091043781,
  27.074847419374755,
  14.01271743883792,
  11.5010

In [8]:
demandas = simulacion_clientes["Volumen por pedidos (m^3)"].tolist()
demandas

[0.0170786,
 0.290464,
 0.00102235,
 0.214957,
 0.001254,
 0.045259,
 0.909348,
 0.0809958,
 0.00642,
 0.02542034,
 0.285304,
 1.46259,
 0.040988,
 0.60705,
 0.4021872499999999,
 0.214957,
 0.0017308,
 0.006235,
 0.00634,
 0.214957,
 0.169222,
 0.1995,
 0.00371844,
 0.00298008,
 1.6789954,
 0.000803,
 0.00772341,
 0.000605,
 0.06,
 0.0018088,
 0.00054,
 0.0009568,
 0.06,
 0.43168,
 0.000611,
 0.0009568,
 1.481515,
 0.021952,
 0.02143112,
 0.49571319,
 0.0374,
 0.004509,
 0.89443947,
 0.00094812,
 0.004511,
 0.027641,
 0.000603,
 0.010501,
 0.62437233,
 0.0001,
 0.0194,
 0.000611,
 0.006729,
 0.04576,
 6.135e-05,
 0.000266,
 0.0192,
 0.37302281,
 1.99605,
 0.9290315,
 0.00105,
 0.21735,
 1.615603,
 3.8509568,
 0.3025,
 0.000603,
 0.008604,
 0.00083164,
 0.001676,
 0.0035158,
 0.217191,
 0.39698326,
 0.0054658,
 0.027193,
 0.537464,
 0.52723387,
 0.005157,
 0.061826,
 0.006235,
 0.1739968,
 0.003171,
 0.109,
 0.00180012,
 0.723,
 0.00093183,
 0.050597,
 0.0035606,
 0.0009568,
 0.00931751

In [9]:
# Suma de la demanda de todos los clientes(en m^3)
simulacion_clientes["Volumen por pedidos (m^3)"].sum()

50.095983700000005

### Algoritmo de busqueda de rutas (_CVRP_)

In [22]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distancias

    data["demands"] = demandas
    data["vehicle_capacities"] = [15.0,15.0,15.0,15.0]
    data["num_vehicles"] = 4
    data["depot"] = 0         
    
    return data

In [24]:
"""Capacited Vehicles Routing Problem (CVRP)."""





def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 282
Route for vehicle 0:
 0 Load(0.0170786) ->  0 Load(0.0170786)
Distance of the route: 0m
Load of the route: 0.0170786

Route for vehicle 1:
 0 Load(0.0170786) ->  0 Load(0.0170786)
Distance of the route: 0m
Load of the route: 0.0170786

Route for vehicle 2:
 0 Load(0.0170786) ->  14 Load(0.4192658499999999) ->  63 Load(4.27022265) ->  15 Load(4.48517965) ->  41 Load(4.48968865) ->  11 Load(5.95227865) ->  65 Load(5.95288165) ->  0 Load(5.95288165)
Distance of the route: 78m
Load of the route: 5.95288165

Route for vehicle 3:
 0 Load(0.0170786) ->  77 Load(0.07890459999999999) ->  53 Load(0.12466459999999999) ->  84 Load(0.12559642999999998) ->  62 Load(1.74119943) ->  7 Load(1.82219523) ->  76 Load(1.82735223) ->  118 Load(2.94872423) ->  166 Load(3.6294143599999997) ->  163 Load(4.37119916) ->  162 Load(4.677246299999999) ->  147 Load(4.678031299999999) ->  155 Load(4.684760299999999) ->  94 Load(4.686078099999999) ->  111 Load(7.668666629999999) ->  122 Load(7.669277629